# Hash Collision Resolution

## Lesson Overview

Collisions are practically unavoidable when using hash functions. However, there are techniques we can use to avoid or **resolve** collisions. All hash table implementations should include a collision resolution algorithm.

For this lesson, we will only consider hash tables that have more buckets than entries. If a hash table has fewer buckets than entries, it is guaranteed to have collisions, and nothing can be done to avoid this. (This is a case of the [Pigeonhole Principle](https://en.wikipedia.org/wiki/Pigeonhole_principle).)

We will implement hash tables using Python lists, where the index is the bucket, and the element is the value. This is a convenient implementation, since the buckets of a hash table are almost always named 0 through $N-1$, where $N$ is the number of buckets, and list indices are named likewise.

### Open addressing

> One of the most commonly used collision resolution is **open addressing**. In this technique, if a value is hashed to a bucket that is already filled, the algorithm systematically tries other buckets until an empty bucket is found. Once an empty bucket is found, the value is put there.

Two of the most common forms of open addressing are:

- Linear probing
- Quadratic probing

### Linear probing

In linear probing, if a value is hashed to an already-filled bucket, we keep looking for an empty bucket by incrementing the bucket number by a constant (usually 1), *modulo the number of buckets*. The modulo operation basically means that if we reach the end of the buckets, we loop back to bucket 0.

> The [modulo](https://en.wikipedia.org/wiki/Modulo_operation) of two numbers returns the remainder after the division of the numbers. $a \ \textrm{mod} \ b$ returns the remainder of the division $a \div b$.

For example, $7 \ \textrm{mod} \ 3 = 1$, since $7 \div 3 = 2 \ \textrm{remainder} \ 1$. In most coding languages, the modulo operation is denoted by a precentage sign `%`, e.g. `a % b`.

See the code below for an implementation of linear probing. There is quite a bit happening here, so take some time to understand the code.

In [ ]:
#persistent
class HashMap:

  def __init__(self, n_buckets):
    """Create an empty list of length n_buckets. This backs the hash table."""
    self.table = [None]*n_buckets

  def linear_resolve(self, item):
    """Add a (bucket, value) pair to an existing hash table.

    Args:
      hash_map: A list where the indices are the hash buckets and the elements
        are the values.
      item: Tuple containing the (bucket, value) to be added to the hash table.
        The tuple is added using linear probing.
    
    Raises:
      ValueError: If the provided bucket value is not within the range of the
        backing list.
    """
    n_buckets = len(self.table)
    bucket, value = item[0], item[1]

    # If the bucket is not in the hash table, raise an error.
    if bucket >= n_buckets:
      raise ValueError(
          "`bucket` parameter %d must be an index of `self.table`." % bucket)

    if self.table[bucket] is None:
      self.table[bucket] = value
    else:
      # Recursively search for the next available bucket.
      self.linear_resolve(((bucket + 1) % n_buckets, value))

### Adding values to a hash table

Let's create a hash table and add some values to it.

In [ ]:
#persistent
# Create a hash table with 10 buckets.
hash_map = HashMap(10)
print(hash_map.table)

# Populate a few buckets.
hash_map.table[1] = "a"
hash_map.table[4] = "b"
hash_map.table[8] = "c"
print(hash_map.table)

---

Now, let's try adding some new values to this table, resolving any collisions using linear probing.

In [ ]:
# Try to add "d" at bucket 2. No resolution necessary.
hash_map.linear_resolve((2, "d"))
print(hash_map.table)

# Try to add "e" at bucket 2. Resolved to bucket 3.
hash_map.linear_resolve((2, "e"))
print(hash_map.table)

# Try to add "f" at bucket 1. Resolved to bucket 5.
hash_map.linear_resolve((1, "f"))
print(hash_map.table)

### Quadratic probing

Instead of successively adding a constant to the hash bucket, quadratic probing adds successive square numbers to the original bucket until an empty bucket is found.

The simplest and most common version of the algorithm is outlined here.

> - **Initialize:** Calculate the hash bucket using the hash function. Call this bucket $H_0$.
> - **Repeat:** If bucket $H_{i-1}$ is filled, calculate the following bucket $H_i = H_0 + i^2 \ \% \ m$ for $i = 1, 2,...$, where $m$ is the number of buckets.
> - **Exit:** As soon as a bucket $H_i$ is empty, fill it with the value.

This is what the algorithm looks like in practice:

0. Calculate the hash bucket of the input value using the hash function.
1. If this bucket is already filled, add $1^2$ to it to get a new bucket.
2. If this following bucket is also filled, add $2^2$ to the *original* hash bucket to get a new bucket.
3. If this next bucket is also filled, add $3^2$ to the *original* bucket to get a new bucket.

The algorithm stops as soon as an empty bucket is found. And if, throughout this process, the hash bucket number exceeds $n$ (where $n$ is the number of buckets in the table), loop around back to bucket 0.

## Question 1

Which of the following statements about hash collision resolution strategies are true? There may be more than one correct response.

**a)** There are exactly two types of hash collision resolution strategies - linear probing and quadratic probing.

**b)** Hash collision resolution strategies help keep the runtime of insertion and retrieval consistent.

**c)** It is possible for a hash collision resolution strategy to fail if there are no available buckets.

**d)** In quadratic probing, we quadruple the bucket number each time to check for an available bucket.

### Solution

The correct answers are **b)** and **c)**.

**a)** These are two common types of open addressing. Open addressing is of the more common hash collision resolution strategies, but many other types of strategies exist.

**d)** We add successive square numbers to the bucket until an empty bucket is found.

## Question 2

You have the following values and hash function.

In [ ]:
values = [17, 44, 13, 25, 5, 4, 26, 6, 31, 15]

def hash_function(input):
  return input**2 % 100

When you hash all of these values, are there any collisions? If so, try and resolve them using linear probing. You may use the `HashMap` class and the associated `linear_resolve` method to add values to a hash map, like so:

```python
hash_map = HashMap(100)

# Add a value to a hash bucket.
hash_map.linear_resolve(bucket, value)
```

What are the final buckets for each value?

In [ ]:
# TODO(you): Write code to calculate the hash bucket for each value.

### Solution

Let's first see what each value is mapped to independently, without any collision resolution. Since the number of values is small, this can be done quickly using a `for` loop and `print`.

In [ ]:
values = [17, 44, 13, 25, 5, 4, 26, 6, 31, 15]

def hash_function(input):
  return input**2 % 100

In [ ]:
for v in values:
  print("hash_function(%d) = %d" % (v, hash_function(v)))

Below are the results.

Input | Hash Bucket
------|------------
17    | 89
44    | 36
13    | 69
25    | 25
5     | 25
4     | 16
26    | 76
6     | 36
31    | 61
15    | 25

- 44 and 6 both map to 36
- 25, 5, and 15 all map to 25

We can resolve these collisions via linear probing.

In [ ]:
# The hash function indicates there are 100 buckets in the hash table.
# This line creates an empty hash table with 100 buckets.
hash_map = HashMap(100)

In [ ]:
# Populate the hash table using the hash function.
# Use linear_resolve to add values and resolve collisions.
for v in values:
  hash_map.linear_resolve((hash_function(v), v))

In [ ]:
for bucket in range(len(hash_map.table)):
  value = hash_map.table[bucket]
  if value is not None:
    print("input: %d, hash bucket: %d" % (value, bucket))

This yields the following final hash table.

Input | Hash Bucket
------|------------
17    | 89
44    | 36
13    | 69
25    | 25
5     | 26
4     | 16
26    | 76
6     | 37
31    | 61
15    | 27

The following three collisions were resolved.
   
1. 5 is hashed to 25 which is already occupied (by 25), so it is resolved to 26 which is empty.
2. 6 is hashed to 36 which is already occupied (by 44), so it is resolved to 37 which is empty.
3. 15 is hashed to 25 which is already occupied (by 25), so it is resolved to 26 which is already occupied (by 5), so it is resolved to 27 which is empty.

## Question 3

Without doing any coding, resolve the following hash table using *quadratic* probing. (Note that there are multiple correct answers to this, since in a collision, you can resolve any of the colliding values.)

Input | Hash Bucket
------|------------
17    | 89
44    | 36
13    | 69
25    | 25
5     | 25
4     | 16
26    | 76
6     | 36
31    | 61
15    | 25

In [ ]:
#freetext

### Solution

Below is the *original* table for reference.

Input | Hash Bucket
------|------------
17    | 89
44    | 36
13    | 69
25    | 25
5     | 25
4     | 16
26    | 76
6     | 36
31    | 61
15    | 25

Let's first identify the collisions.

- 44 and 6 both map to 36
- 25, 5, and 15 all map to 25

These can be resolved as follows.

1. 5 is hashed to 25 which is already occupied (by 25), so it is resolved to 26 ($25+1^2$) which is empty.
2. 6 is hashed to 36 which is already occupied (by 44), so it is resolved to 37 ($36+1^2$) which is empty.
3. 15 is hashed to 25 which is already occupied (by 25), so it is resolved to 26 ($25+1^2$) which is already occupied (by 5), so it is resolved to 29 ($25+2^2$) which is empty.

Input | Hash Bucket
------|------------
17    | 89
44    | 36
13    | 69
25    | 25
5     | 26
4     | 16
26    | 76
6     | 37
31    | 61
15    | 29

## Question 4

Without writing any code, resolve the following hash table using linear probing. There are 20 buckets in the table (labelled 0-19). Assume that input values are hashed in the order in which they appear in the table. This means that rows that come first in the table have precedence to keep their original hash bucket. (For example, when resolving bucket 15, you should allow input 3 to keep this bucket and find a new bucket for input 6.)

Input | Hash Bucket
------|------------
3     | 15
9     | 0
8     | 3
2     | 19
6     | 15
0     | 7
5     | 16
7     | 19

In [ ]:
#freetext

### Solution

Below is the *original* table for reference.

Input | Hash Bucket
------|------------
3     | 15
9     | 0
8     | 3
2     | 19
6     | 15
0     | 7
5     | 16
7     | 19

**Step 1: Resolving bucket 15**
   
Both 3 and 6 hash to the same bucket, 15. We will resolve for the input value that came later, which is 6. To resolve the collision, we add 1 to the original bucket value of 15 to get 16, which is already filled (by 5). So we add 1 again to get 17, which is empty. Therefore the resolved hash bucket for 6 is 17. The updated table is below.

Input | Hash Bucket
------|------------
3     | 15
9     | 0
8     | 3
2     | 19
6     | 17
0     | 7
5     | 16
7     | 19

**Step 2: Resolving bucket 19**
   
The next collision is in bucket 19, for inputs 2 and 7. Again, we will resolve for the input that came later, which is 7. We start by adding 1 to 19 which gives 0, which is already filled (by 9). *Since there are 20 buckets, all additions are modulo 20, so $19+1=0$.* Another way to think about this is that there is no bucket 20, so we loop back around to bucket 0. So we add 1 again to get 1, which is empty. Therefore the resolved hash bucket for 7 is 1. The final hash table is below.

Input | Hash Bucket
------|------------
3     | 15
9     | 0
8     | 3
2     | 19
6     | 17
0     | 7
5     | 16
7     | 1

## Question 5

Without writing any code, resolve the following hash table using *quadratic* probing. There are 20 buckets in the table (labelled 0-19). Assume that input values are hashed in the order in which they appear in the table. This means that rows that come first in the table have precedence to keep their original hash bucket. (For example, when resolving bucket 15, you should allow input 3 to keep this bucket and find a new bucket for input 6.)

Input | Hash Bucket
------|------------
3     | 15
9     | 0
8     | 3
2     | 19
6     | 15
0     | 7
5     | 16
7     | 19

In [ ]:
#freetext

### Solution

Below is the original hash table, for reference.

Input | Hash Bucket
------|------------
3     | 15
9     | 0
8     | 3
2     | 19
6     | 15
0     | 7
5     | 16
7     | 19

**Step 1: Resolving bucket 15**
   
We try to resolve this by adding $1^2$ to 15 which is 16, which is already filled (by 5). So we add $2^2$ to 15 which is 19, which is already filled by 7. So we add $3^2$ to 15 (modulo 20) to get $24 \ \textrm{mod} \ 20 = 4$, which is empty. Therefore the resolved hash bucket for 6 is 4.

Input | Hash Bucket
------|------------
3     | 15
9     | 0
8     | 3
2     | 19
6     | 4
0     | 7
5     | 16
7     | 19

**Step 2: Resolving bucket 19**
   
We try to resolve this by adding $1^2$ to 19 (modulo 20) to get $20 \ \textrm{mod} \ 20 = 0$, which is already filled (by 9). So we add $2^2$ to 19 (modulo 20) to get $23 \ \textrm{mod} \ 20 = 3$, which is already filled (by 8). So we add $3^2$ to 19 (modulo 20) to get $28 \ \textrm{mod} \ 20 = 8$, which is empty. Therefore the resolved hash bucket for 7 is 8. The final output table is below.

Input | Hash Bucket
------|------------
3     | 15
9     | 0
8     | 3
2     | 19
6     | 17
0     | 7
5     | 16
7     | 8

## Question 6

Add a `quadratic_resolve` method to the code snippet in the Lesson Overview. The code for `linear_resolve` is below; it may be useful for reference.

This problem can be solved iteratively (using a `for` or `while` loop) or recursively (calling the function within itself), but the recursive solution may require adding another input parameter.

In [ ]:
class HashMap:

  def __init__(self, n_buckets):
    """Create an empty list of length n_buckets. This is the backing table."""
    self.table = [None]*n_buckets

  def linear_resolve(self, item):
    """Add a (bucket, value) pair to an existing hash table.

    Args:
      hash_map: A list where the indices are the hash buckets and the elements
        are the values.
      item: Tuple containing the (bucket, value) to be added to the hash table.
        The tuple is added using linear probing.
    """
    n_buckets = len(self.table)
    bucket, value = item[0], item[1]

    # If the bucket is not in the hash table, raise an error.
    if bucket >= n_buckets:
      raise ValueError(
          "bucket parameter %d must be an index of self.table." % bucket)

    if self.table[bucket] is None:
      self.table[bucket] = value
    else:
      # Recursively search for the next available bucket.
      self.linear_resolve(((bucket + 1) % n_buckets, value))

  def quadratic_resolve(self, item):
    """Add a (bucket, value) pair to an existing hash table.

    Args:
      hash_map: A list where the indices are the hash buckets and the elements
        are the values.
      item: Tuple containing the (bucket, value) to be added to the hash table.
        The tuple is added using quadratic probing.
    """
    # TODO(you): Implement
    print("This method has not been implemented.")

### Unit Tests

Run the following cell to check your answer against some unit tests.

In [ ]:
hash_map = HashMap(10)

hash_map.table[1] = "a"
hash_map.table[4] = "b"
hash_map.table[8] = "c"

# Try to add "d" at bucket 9. No resolution necessary.
hash_map.quadratic_resolve((9, "d"))
print(hash_map.table)
# Should print: [None, 'a', None, None, 'b', None, None, None, 'c', 'd']

# Try to add "e" at bucket 9. Resolved to bucket 0.
hash_map.quadratic_resolve((9, "e"))
print(hash_map.table)
# Should print: ['e', 'a', None, None, 'b', None, None, None, 'c', 'd']

# Try to add "f" at bucket 8. Resolved to bucket 2.
hash_map.quadratic_resolve((8, "f"))
print(hash_map.table)
# Should print: ['e', 'a', 'f', None, 'b', None, None, None, 'c', 'd']

### Solution

This is just one possible solution. There are several other potential solutions, for example using a `while` loop.

In [ ]:
class HashMap:

  def __init__(self, n_buckets):
    """Create an empty list of length n_buckets. This is the backing table."""
    self.table = [None]*n_buckets

  def linear_resolve(self, item):
    """Add a (bucket, value) pair to an existing hash table.

    Args:
      hash_map: A list where the indices are the hash buckets and the elements
        are the values.
      item: Tuple containing the (bucket, value) to be added to the hash table.
        The tuple is added using linear probing.
    """
    n_buckets = len(self.table)
    bucket, value = item[0], item[1]

    # If the bucket is not in the hash table, raise an error.
    if bucket >= n_buckets:
      raise ValueError(
          "bucket parameter %d must be an index of self.table." % bucket)

    if self.table[bucket] is None:
      self.table[bucket] = value
    else:
      # Recursively search for the next available bucket.
      self.linear_resolve(((bucket + 1) % n_buckets, value))

  def quadratic_resolve(self, item, step=1):
    """Add a (bucket, value) pair to an existing hash table.

    Args:
      hash_map: A list where the indices are the hash buckets and the elements
        are the values.
      item: Tuple containing the (bucket, value) to be added to the hash table.
        The tuple is added using quadratic probing.
      step: This is a helper parameter, and should not be manually configured.
    """
    n_buckets = len(self.table)
    bucket, value = item[0], item[1]

    # If the bucket is not in the hash table, raise an error.
    if bucket >= n_buckets:
      raise ValueError(
          "bucket parameter %d must be an index of self.table." % bucket)

    if self.table[bucket] is None:
      self.table[bucket] = value
    else:
      # Recursively search for the next available bucket.
      # Increment the step parameter by one each time.
      self.quadratic_resolve(((bucket + step**2) % n_buckets, value, step+1))

## Question 7

Your coworker has sent you some code to review. It uses quadratic probing to resolve collisions in a hash table. It uses a `while` loop instead of a recursive algorithm for `quadratic_resolve`. The code is currently not running, and you are seeing an `IndexError` in the unit test log. Your coworker has asked you to identify what is causing the index error.

In [ ]:
class HashMap:

  def __init__(self, n_buckets):
    """Create an empty list of length n_buckets. This is the backing table."""
    self.table = [None]*n_buckets

  def quadratic_resolve(self, item):
    # TODO(you): Fix the code so that it no longer raises an error.
    """Add a (bucket, value) pair to an existing hash table.

    Args:
      hash_map: A list where the indices are the hash buckets and the elements
        are the values.
      item: Tuple containing the (bucket, value) to be added to the hash table.
        The tuple is added using quadratic probing.
    """
    n_buckets = len(self.table)
    bucket, value = item[0], item[1]

    # If the bucket is not in the hash table, raise an error.
    if bucket >= n_buckets:
      raise ValueError(
          "bucket parameter %d must be an index of self.table." % bucket)

    new_bucket = bucket
    counter = 1
    # Keep looping until a free spot in the hash table is found.
    while self.table[new_bucket] is not None:
      new_bucket += counter**2
      counter += 1

    self.table[new_bucket] = value

The following code results in an `IndexError`.

In [ ]:
hash_map = HashMap(10)

hash_map.table[1] = "a"
hash_map.table[4] = "b"
hash_map.table[8] = "c"

# Try to add "d" at bucket 9. No resolution necessary.
hash_map.quadratic_resolve((9, "d"))
print(hash_map.table)

# Try to add "e" at bucket 9. Resolved to bucket 0.
hash_map.quadratic_resolve((9, "e"))
print(hash_map.table)

# Try to add "f" at bucket 8. Resolved to bucket 2.
hash_map.quadratic_resolve((8, "f"))
print(hash_map.table)

### Unit Tests

Run the following cell to check your answer against some unit tests.

In [ ]:
hash_map = HashMap(10)

hash_map.table[1] = "a"
hash_map.table[4] = "b"
hash_map.table[8] = "c"

# Try to add "d" at bucket 9. No resolution necessary.
hash_map.quadratic_resolve((9, "d"))
# Should not raise error

# Try to add "e" at bucket 9. Resolved.
hash_map.quadratic_resolve((9, "e"))
# Should not raise error

# Try to add "f" at bucket 8. Resolved.
hash_map.quadratic_resolve((8, "f"))
# Should not raise error

### Solution

The cause of the error is that, within the `while` loop, the addition of `counter**2` to `new_hash` needs to be performed modulo the number of buckets. Without this, the code is trying to put a value into a bucket value that should not exist in the hash table.

This is fixed by simply making the addition modulo `n_buckets`. This can be done by either:

- Adding the line `new_bucket = new_bucket % n_buckets` 
- Changing the existing line to `new_bucket = (new_bucket + counter**2) % n_buckets`

In [ ]:
class HashMap:

  def __init__(self, n_buckets):
    """Create an empty list of length n_buckets. This is the backing table."""
    self.table = [None]*n_buckets

  def quadratic_resolve(self, item):
    """Add a (bucket, value) pair to an existing hash table.

    Args:
      hash_map: A list where the indices are the hash buckets and the elements
        are the values.
      item: Tuple containing the (bucket, value) to be added to the hash table.
        The tuple is added using quadratic probing.
    """
    n_buckets = len(self.table)
    bucket, value = item[0], item[1]

    # If the bucket is not in the hash table, raise an error.
    if bucket >= n_buckets:
      raise ValueError(
          "bucket parameter %d must be an index of self.table." % bucket)

    new_bucket = bucket
    counter = 1
    # Keep looping until a free spot in the hash table is found.
    while self.table[new_bucket] is not None:
      # The following line changed to be modulo n_buckets.
      new_bucket = (new_bucket + counter**2) % n_buckets
      counter += 1

    self.table[new_bucket] = value

Now, the same code runs without an `IndexError`.

In [ ]:
hash_map = HashMap(10)

hash_map.table[1] = "a"
hash_map.table[4] = "b"
hash_map.table[8] = "c"

# Try to add "d" at bucket 9. No resolution necessary.
hash_map.quadratic_resolve((9, "d"))
print(hash_map.table)

# Try to add "e" at bucket 9. Resolved to bucket 0.
hash_map.quadratic_resolve((9, "e"))
print(hash_map.table)

# Try to add "f" at bucket 8. Resolved to bucket 2.
hash_map.quadratic_resolve((8, "f"))
print(hash_map.table)

## Question 8

Your fix worked! The code now runs. However, your coworker has noticed that the unit tests do not pass. These are unit tests that you wrote based on your existing knowledge of quadratic probing. (For this problem, you can assume that the unit tests are correctly implemented.) Your coworker is having trouble figuring out where they are going wrong, and they have asked you to review the code.

In [ ]:
class HashMap:

  def __init__(self, n_buckets):
    """Create an empty list of length n_buckets. This is the backing table."""
    self.table = [None]*n_buckets

  def quadratic_resolve(self, item):
    # TODO(you): Find the bug that causes the unit tests to fail
    """Add a (bucket, value) pair to an existing hash table.

    Args:
      hash_map: A list where the indices are the hash buckets and the elements
        are the values.
      item: Tuple containing the (bucket, value) to be added to the hash table.
        The tuple is added using quadratic probing.
    """
    n_buckets = len(self.table)
    bucket, value = item[0], item[1]

    # If the bucket is not in the hash table, raise an error.
    if bucket >= n_buckets:
      raise ValueError(
          "bucket parameter %d must be an index of self.table." % bucket)

    new_bucket = bucket
    counter = 1
    # Keep looping until a free spot in the hash table is found.
    while self.table[new_bucket] is not None:
      # The following line changed to be modulo n_buckets.
      new_bucket = (new_bucket + counter**2) % n_buckets
      counter += 1

    self.table[new_bucket] = value

### Hint

In the following example, the existing algorithm resolves `"f"` to bucket 3, but it should be resolved to bucket 2.

In [ ]:
hash_map = HashMap(10)

hash_map.table[1] = "a"
hash_map.table[4] = "b"
hash_map.table[8] = "c"

hash_map.quadratic_resolve((9, "d"))
print(hash_map.table)

hash_map.quadratic_resolve((9, "e"))
print(hash_map.table)

hash_map.quadratic_resolve((8, "f"))
print(hash_map.table)

### Unit Tests

Run the following cell to check your answer against some unit tests.

In [ ]:
hash_map = HashMap(10)

hash_map.table[1] = "a"
hash_map.table[4] = "b"
hash_map.table[8] = "c"

# Try to add "d" at bucket 9. No resolution necessary.
hash_map.quadratic_resolve((9, "d"))
print(hash_map.table)
# Should print: [None, 'a', None, None, 'b', None, None, None, 'c', 'd']

# Try to add "e" at bucket 9. Resolved to bucket 0.
hash_map.quadratic_resolve((9, "e"))
print(hash_map.table)
# Should print: ['e', 'a', None, None, 'b', None, None, None, 'c', 'd']

# Try to add "f" at bucket 8. Resolved to bucket 2.
hash_map.quadratic_resolve((8, "f"))
print(hash_map.table)
# Should print: ['e', 'a', 'f', None, 'b', None, None, None, 'c', 'd']

### Solution

This is a subtle bug. Not fixing this won't break anything, but if you don't fix it, your coworker's code will not implement the same algorithm as was outlined in the Lesson Overview.

Instead of adding `counter**2` to the `new_bucket` value, the code should be adding it to the *original* hash value `bucket`.

```python
new_bucket = (bucket + counter^2) % n_buckets
```

In [ ]:
class HashMap:

  def __init__(self, n_buckets):
    """Create an empty list of length n_buckets. This is the backing table."""
    self.table = [None]*n_buckets

  def quadratic_resolve(self, item):
    """Add a (bucket, value) pair to an existing hash table.

    Args:
      hash_map: A list where the indices are the hash buckets and the elements
        are the values.
      item: Tuple containing the (bucket, value) to be added to the hash table.
        The tuple is added using quadratic probing.
    """
    n_buckets = len(self.table)
    bucket, value = item[0], item[1]

    # If the bucket is not in the hash table, raise an error.
    if bucket >= n_buckets:
      raise ValueError(
          "bucket parameter %d must be an index of self.table." % bucket)

    new_bucket = bucket
    counter = 1
    # Keep looping until a free spot in the hash table is found.
    while self.table[new_bucket] is not None:
      # The following line changed to use bucket instead of new_bucket.
      new_bucket = (bucket + counter**2) % n_buckets
      counter += 1

    self.table[new_bucket] = value